# Single Subject GLM Analysis using the Localizer dataset

In this lab, extend your work from the previous lab to work with 3D brain data. The preprocessed data of the first subject in the localizer study is included.

In [2]:
import os
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import nibabel as nib
import sys
sys.path.insert(0, './src')
import load_data
import design_matrix
import estimate_beta
import contrast
import visualization
import clusters
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy.ndimage

Load data and mask.

In [3]:
data, mask = load_data.load_data('preprocessed_fMRI_bold.nii.gz','sub-S01_mask.nii.gz')

Design matrix.

In [4]:
events, labels = design_matrix.get_localizer_design_data()
dm = design_matrix.create_design_matrix(events, labels, 2.4, 128)
funcs = [
    lambda dm : design_matrix.convolve_hrf(dm, 2),
    lambda dm : design_matrix.add_intercept(dm),
    lambda dm : design_matrix.add_poly(dm, degrees = 2),
    lambda dm : design_matrix.add_poly(dm, degrees = 3),
    lambda dm : design_matrix.add_poly(dm, degrees = 2, flipped = True),
    lambda dm : design_matrix.add_poly(dm, degrees = 3, flipped = True),  
    lambda dm : design_matrix.add_cosine(dm, 1, 1),
    lambda dm : design_matrix.add_cosine(dm, 2, 1),    
    lambda dm : design_matrix.add_cosine(dm, 3, 1),        
]

dm = design_matrix.add_many(dm, funcs)

Esitmate beta.

In [5]:
bm = estimate_beta.compute_beta_map(data, dm.values)

Generate Contrast

In [6]:
ct = contrast.generateContrast(dm, 'motor', 'sentence+subtraction')
cs = contrast.compute_contrast(bm, ct)
masked_contrast = visualization.getMaskedContrast(cs, mask.get_fdata(),0.1)
masked_forvis = visualization.getMasked(cs, mask.get_fdata(),0.1)

Visualize Contrast

In [7]:
xm,ym,zm= visualization.find_most_active_slices_xyz(masked_contrast)
print(xm,ym,zm)
@interact(x=(0,data.get_fdata().shape[0]), y=(0,data.get_fdata().shape[1]), z=(0,data.get_fdata().shape[2]))
def interact_with_voxels(x=xm, y=ym, z=zm):
    visualization.visualiza_contrast_withLocation_Interactive(masked_forvis, x,y,z)

26 56 5


interactive(children=(IntSlider(value=26, description='x', max=53), IntSlider(value=56, description='y', max=6…

Convolve with Gassuian Filter

In [8]:
ct_cluster = masked_contrast
ct_cluster = clusters.convolveFilter(ct_cluster, 0.2)
ct_clusterforvis = masked_forvis
ct_clusterforvis = clusters.convolveFilter(ct_clusterforvis, 0.2)

xm,ym,zm= visualization.find_most_active_slices_xyz(ct_cluster)
print(xm,ym,zm)
@interact(x=(0,data.get_fdata().shape[0]), y=(0,data.get_fdata().shape[1]), z=(0,data.get_fdata().shape[2]))
def interact_with_voxels(x=xm, y=ym, z=zm):
    visualization.visualiza_contrast_withLocation_Interactive(ct_clusterforvis, x,y,z)

26 56 5


interactive(children=(IntSlider(value=26, description='x', max=53), IntSlider(value=56, description='y', max=6…

Find Clusters and Centers

In [9]:
ct_clustered, num_clusters = clusters.label_clusters(ct_cluster)
centers = scipy.ndimage.measurements.center_of_mass(ct_cluster, ct_clustered, np.arange(num_clusters))

print(num_clusters, centers)


39 [(nan, nan, nan), (10.364466607385289, 20.602787345327858, 13.778330706211431), (8.806123704241829, 24.79876267028007, 32.60489478872541), (10.007844749899185, 50.32272533391457, 15.687742532246597), (10.597506887038351, 33.571779945128505, 19.542767284589132), (10.000000000000002, 43.32349015528426, 31.655836231383418), (25.38006013244358, 30.82691052892176, 10.470500589088616), (14.587996896281652, 55.02817838003409, 4.555175518357329), (15.116977126767976, 41.934600404896905, 36.848254491547564), (13.999999999999995, 12.999999999999991, 11.999999999999993), (14.317567053934411, 44.16219904840007, 30.16324099922387), (15.0, 26.999999999999993, 7.0), (17.272729417122292, 22.867345477845884, 23.2227795120316), (18.501907876734467, 3.4980921232655273, 14.501907876734467), (28.007624025506313, 9.009947493370351, 11.633960668793204), (18.0, 8.0, 26.000000000000007), (21.86381106208617, 57.544144646688366, 13.780410394608834), (19.50372133331208, 57.999999999999986, 25.000000000000004),

/Users/hanxu/miniconda3/lib/python3.6/site-packages/scipy/ndimage/measurements.py:1359: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]


In [20]:
@interact(c=(0,len(centers)-1))
def clustersVis(c=2):
    datavis = clusters.get_clusterforvis(ct_clusterforvis, ct_clustered, c)
    @interact(x=(0,data.get_fdata().shape[0]), y=(0,data.get_fdata().shape[1]), z=(0,data.get_fdata().shape[2]))
    def plot_clusters(x= 10, y =21, z =14):
        visualization.visualiza_contrast_withLocation_Interactive(datavis, x,y,z)

interactive(children=(IntSlider(value=2, description='c', max=38), Output()), _dom_classes=('widget-interact',…